# 提高模型泛化能力

经过前面的步骤,现在应该已经有一个可以过拟合的模型了,接下来要做的事情是找到欠拟合/过拟合的边界,使得模型泛化能力 MAX.


## 数据集

前文的流形假说,深度学习的泛化能力与样本之间插值有关.显而易见的如果样本之间足够平滑,模型训练后的泛化能力要更好.

必须确保正在使用的数据集是合适的.在数据上花费等量资源,其回报通常比开发更好模型的回报更高.

- 尽可能确保样本数量,更多的数据 == 更好的模型.数学之美-32大数据威力一节提到了一个例子,2005年 google 的机器翻译模型在 NIST 封闭数据集高于第二名 5%,开放测试集高于 17%.当时 5% 的提高要 5-10 年,google 是如何做到的?当别的模型只是 4 元模型时,Google使用了成千甚至上万倍的数据,训练出了 6 元模型.这是堆砌数据的威力.
- 减少标签错误,异常值会使得过拟合提前.
- 清理数据,处理缺失值.(下一节)
- 如果数据特征及其繁杂,需要进行取舍.->特征工程,选择真正有用的特征.


## 特征工程

特征工程是在数据进入模型前,人工筛选,转换清理数据.

在深度学习出现以前,特征工程是机器学习,极为重要的一环.几乎决定了成败.深度学习出现以后,特征工程的很大一部分工作被神经网络自动化了.但是这并不意味着特征工程无用.

- 以现在的深度学习能力,还不能指望机器学习能够完全从任意数据上学习.
- 选择好的特征意味着更小的模型,更小的资源花费.
- 特征工程意味着更少的数据可以解决问题,在样本数量不足时非常有用.

![clock_diagram](./clock_diagram.png)

以钟表的数据为例

- 原始数据是图像数据,不加处理直接输入模型,模型需要 RNN + 大量时间/计算资源
- 以坐标表示输入模型,输入减少成了 2d 张量,几乎成了线性拟合.
- 干脆以中心为原点,换用极坐标表示.都不需要机器学习了.

以更简单的方式表述问题,这就是特征工程.

个人愚见: 包含特征工程在内的整个深度学习流程,就是一个从数据找规律,提取到足以解决问题的特征的过程.特征工程找到的规律多一分,接下来深度学习需要学习的部分就少了一分,训练需要的资源也就少一分.


## 提前终止

在训练中找到过拟合/欠拟合准确的边界,这是提供模型泛化能力最有效的手段之一.

上一章的例子中.我们不断训练模型直到明显的过拟合,综合数据找到过拟合/欠拟合边界的轮次.再开启重新的训练,迭代到边界.但是实际上并不需要这样,只要保存每次迭代的模型,在模型出现明显退化时,回滚到之前更好的模型即可.

以上就是大致的提取终止(early stopping),在 keras 中使用 `EarlyStopping` 回调,我们会在第 7 章看见.
